In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT
*
FROM `perceptive-ivy-290216.f1_api.race_lap_time`  A
# WHERE A.Year=2024
# AND A.GP="Monaco Grand Prix"
# AND A.DRIVER='HAM'
ORDER BY LapNumber, LapStartTime
"""
track3=pd.read_gbq(query2,project_id,dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_12969/2934139378.py:10: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  track3=pd.read_gbq(query2,project_id,dialect='standard')


In [236]:
track2=track3[(track3["GP"]=='Hungarian Grand Prix')&(track3["Year"]==2024)]

In [237]:
track2.head()
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]

In [238]:
track2['LapTime']= pd.to_timedelta(track2["LapTime"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_12969/4173085553.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [239]:
# track2['LapTime'].dt.total_seconds().min()*1.07
track2=track2[track2['LapNumber']!=1.0]
track2.loc[:, "LapTime (s)"] = track2["LapTime"].dt.total_seconds()

In [240]:
quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
quicklaps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s)
20394,0 days 00:57:56.002000,PIA,81,0 days 00:01:24.038000,2.0,1.0,NaT,NaT,0 days 00:00:29.924000,0 days 00:00:30.425000,0 days 00:00:23.689000,0 days 00:57:01.888000,0 days 00:57:32.313000,0 days 00:57:56.002000,264.0,231.0,244.0,285.0,True,MEDIUM,2.0,True,McLaren,0 days 00:56:31.964000,NaT,1,1.0,False,,False,True,2024,Hungarian Grand Prix,84.038
20395,0 days 00:57:57.580000,VER,1,0 days 00:01:24.026000,2.0,1.0,NaT,NaT,0 days 00:00:29.907000,0 days 00:00:30.574000,0 days 00:00:23.545000,0 days 00:57:03.478000,0 days 00:57:34.052000,0 days 00:57:57.597000,NaN,235.0,244.0,287.0,True,MEDIUM,2.0,True,Red Bull Racing,0 days 00:56:33.554000,NaT,1,2.0,False,,False,True,2024,Hungarian Grand Prix,84.026
20396,0 days 00:57:58.847000,NOR,4,0 days 00:01:24.240000,2.0,1.0,NaT,NaT,0 days 00:00:30.091000,0 days 00:00:30.380000,0 days 00:00:23.769000,0 days 00:57:04.740000,0 days 00:57:35.120000,0 days 00:57:58.889000,272.0,235.0,245.0,291.0,True,MEDIUM,2.0,True,McLaren,0 days 00:56:34.607000,NaT,1,3.0,False,,False,True,2024,Hungarian Grand Prix,84.240
20397,0 days 00:58:00.291000,HAM,44,0 days 00:01:24.832000,2.0,1.0,NaT,NaT,0 days 00:00:30.357000,0 days 00:00:30.720000,0 days 00:00:23.755000,0 days 00:57:05.831000,0 days 00:57:36.551000,0 days 00:58:00.306000,268.0,236.0,242.0,286.0,True,MEDIUM,2.0,True,Mercedes,0 days 00:56:35.459000,NaT,1,4.0,False,,False,True,2024,Hungarian Grand Prix,84.832
20398,0 days 00:58:01.512000,LEC,16,0 days 00:01:25.076000,2.0,1.0,NaT,NaT,0 days 00:00:30.320000,0 days 00:00:30.752000,0 days 00:00:24.004000,0 days 00:57:06.766000,0 days 00:57:37.518000,0 days 00:58:01.522000,277.0,232.0,243.0,290.0,True,MEDIUM,2.0,True,Ferrari,0 days 00:56:36.436000,NaT,1,5.0,False,,False,True,2024,Hungarian Grand Prix,85.076


In [241]:
transformed_laps = quicklaps.copy()
transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

# order the team from the fastest (lowest median lap time) to slower
team_order = (
    transformed_laps[["Team", "LapTime (s)"]].groupby("Team").median()["LapTime (s)"].sort_values().index
)
print(team_order)

Index(['Mercedes', 'Red Bull Racing', 'McLaren', 'Ferrari', 'Aston Martin',
       'RB', 'Haas F1 Team', 'Williams', 'Kick Sauber', 'Alpine'],
      dtype='object', name='Team')


In [242]:
transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('median')
transformed_laps.tail()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s),median
577073,0 days 02:33:46.999000,PER,11,0 days 00:01:23.447000,70.0,3.0,NaT,NaT,0 days 00:00:29.862000,0 days 00:00:30.116000,0 days 00:00:23.469000,0 days 02:32:53.421000,0 days 02:33:23.537000,0 days 02:33:47.006000,NaN,244.0,249.0,NaN,False,MEDIUM,23.0,True,Red Bull Racing,0 days 02:32:23.552000,NaT,1,7.0,False,,False,True,2024,Hungarian Grand Prix,83.447,83.416
577074,0 days 02:33:49.575000,RUS,63,0 days 00:01:24.063000,70.0,3.0,NaT,NaT,0 days 00:00:29.633000,0 days 00:00:30.728000,0 days 00:00:23.702000,0 days 02:32:55.150000,0 days 02:33:25.878000,0 days 02:33:49.580000,285.0,244.0,251.0,316.0,False,HARD,17.0,True,Mercedes,0 days 02:32:25.512000,NaT,1,8.0,False,,False,True,2024,Hungarian Grand Prix,84.063,83.319
577077,0 days 02:34:24.466000,TSU,22,0 days 00:01:24.741000,70.0,2.0,NaT,NaT,0 days 00:00:30.269000,0 days 00:00:30.460000,0 days 00:00:24.012000,0 days 02:33:29.996000,0 days 02:34:00.456000,0 days 02:34:24.468000,NaN,236.0,246.0,290.0,False,HARD,41.0,True,RB,0 days 02:32:59.725000,NaT,1,9.0,False,,False,True,2024,Hungarian Grand Prix,84.741,84.412
577078,0 days 02:34:25.183000,STR,18,0 days 00:01:24.443000,70.0,3.0,NaT,NaT,0 days 00:00:29.840000,0 days 00:00:30.496000,0 days 00:00:24.107000,0 days 02:33:30.580000,0 days 02:34:01.076000,0 days 02:34:25.183000,NaN,241.0,251.0,307.0,False,HARD,26.0,False,Aston Martin,0 days 02:33:00.740000,NaT,1,10.0,False,,False,True,2024,Hungarian Grand Prix,84.443,84.387
577079,0 days 02:34:29.667000,ALO,14,0 days 00:01:25.448000,70.0,3.0,NaT,NaT,0 days 00:00:30.062000,0 days 00:00:30.965000,0 days 00:00:24.421000,0 days 02:33:34.284000,0 days 02:34:05.249000,0 days 02:34:29.670000,275.0,238.0,246.0,NaN,False,HARD,34.0,False,Aston Martin,0 days 02:33:04.219000,NaT,1,11.0,False,,False,True,2024,Hungarian Grand Prix,85.448,84.387


In [243]:
fig=px.box(
    transformed_laps.sort_values(by=["median","LapNumber"]),
    x="Team",
    y="LapTime (s)",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace Plot for the {} {}</b>".format(year,gp),
    # palette=team_palette,
    # whiskerprops=dict(color="white"),
    # boxprops=dict(edgecolor="white"),
    # medianprops=dict(color="grey"),
    # capprops=dict(color="white"),
    height=700, width=1200,

color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)
fig.update_traces(opacity=1)
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    margin=dict(l=60, r=5, t=50, b=60),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.show()

In [244]:
fig.write_html("{} {} Team Pace Plot.html".format(year,gp))

In [245]:
#Get MINIMUM OF THE MEDIAN LAP TIMES

fastest_lap = transformed_laps["median"].min()
transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
transformed_laps_relative_median=transformed_laps[["Team","Delta","Delta_Percent"]].drop_duplicates()
transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
transformed_laps_relative_median

,Team,Delta,Delta_Percent
20397,Mercedes,0.0000,100.0%
20395,Red Bull Racing,0.0970,100.116%
20394,McLaren,0.1370,100.164%
20398,Ferrari,0.1700,100.204%
52585,Aston Martin,1.0680,101.282%
52590,RB,1.0930,101.312%
41910,Haas F1 Team,1.4175,101.701%
73818,Williams,1.4570,101.749%
73815,Kick Sauber,1.6605,101.993%
73824,Alpine,1.7420,102.091%


In [246]:
for index, row in transformed_laps_relative_median.iterrows():
  if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
    transformed_laps_relative_median.loc[index, 'Delta']=0.001
transformed_laps_relative_median

,Team,Delta,Delta_Percent
20397,Mercedes,0.0010,100.0%
20395,Red Bull Racing,0.0970,100.116%
20394,McLaren,0.1370,100.164%
20398,Ferrari,0.1700,100.204%
52585,Aston Martin,1.0680,101.282%
52590,RB,1.0930,101.312%
41910,Haas F1 Team,1.4175,101.701%
73818,Williams,1.4570,101.749%
73815,Kick Sauber,1.6605,101.993%
73824,Alpine,1.7420,102.091%


In [247]:
fig=px.bar(
    transformed_laps_relative_median,
    x="Team",
    y="Delta",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace (Median Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent",
    height=700, width=1200,
color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)

fig.update_traces(textposition='outside')

fig.update_traces(opacity=0.85)

fig.update_traces( marker_line_color='white',marker_line_width=1.5)

fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig.update_yaxes(range=[2.1, 0])

# fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig.update_xaxes(title_text='')      # Remove axis titles
fig.update_yaxes(title_text='')

fig.update_xaxes(
        title_standoff = 75
)

fig.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig.show()

In [248]:
fig.write_html("{} {} Team Median Plot.html".format(year,gp))

In [249]:
#Get MINIMUM OF ALL LAP TIMES
transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('min')

fastest_lap_all = transformed_laps["min_all"].min()
transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
transformed_laps_relative_min_all=transformed_laps[["Team","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
transformed_laps_relative_min_all

,Team,Delta_Min_All,Delta_Percent_All
20397,Mercedes,0.000,100.0%
73818,Williams,0.256,100.319%
20395,Red Bull Racing,0.603,100.751%
20398,Ferrari,1.136,101.415%
73824,Alpine,1.305,101.625%
20394,McLaren,1.407,101.752%
52585,Aston Martin,2.033,102.532%
52590,RB,2.335,102.908%
73815,Kick Sauber,2.487,103.097%
41910,Haas F1 Team,3.156,103.93%


In [250]:
for index, row in transformed_laps_relative_min_all.iterrows():
  if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
    transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
transformed_laps_relative_min_all

,Team,Delta_Min_All,Delta_Percent_All
20397,Mercedes,0.001,100.0%
73818,Williams,0.256,100.319%
20395,Red Bull Racing,0.603,100.751%
20398,Ferrari,1.136,101.415%
73824,Alpine,1.305,101.625%
20394,McLaren,1.407,101.752%
52585,Aston Martin,2.033,102.532%
52590,RB,2.335,102.908%
73815,Kick Sauber,2.487,103.097%
41910,Haas F1 Team,3.156,103.93%


In [251]:
fig=px.bar(
    transformed_laps_relative_min_all,
    x="Team",
    y="Delta_Min_All",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace (Fastest Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent_All",
    height=700, width=1200,
color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)

fig.update_traces(textposition='outside')

fig.update_traces(opacity=0.85)

fig.update_traces( marker_line_color='white',marker_line_width=1.5)

fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig.update_yaxes(range=[3.3, 0])

# fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig.update_xaxes(title_text='')      # Remove axis titles
fig.update_yaxes(title_text='')

fig.update_xaxes(
        title_standoff = 75
)

fig.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.update_yaxes(ticksuffix = "  ")
fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig.show()

In [252]:
fig.write_html("{} {} Team Fastest Lap Plot.html".format(year,gp))

In [253]:
# #Get AVERAGE OF ALL LAP TIMES
# transformed_laps['avg_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform(np.mean)

# fastest_lap_avg = transformed_laps["avg_all"].min()
# transformed_laps['Delta_Avg_All']= transformed_laps["avg_all"] - (fastest_lap_avg)
# transformed_laps['Delta_Percent_Avg']= (transformed_laps["avg_all"]/(fastest_lap_avg)-1)*100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].round(3)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].astype(str)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+"%"
# transformed_laps_relative_avg_all=transformed_laps[["Team","Delta_Avg_All","Delta_Percent_Avg"]].drop_duplicates()
# transformed_laps_relative_avg_all=transformed_laps_relative_avg_all.sort_values("Delta_Avg_All")
# transformed_laps_relative_avg_all

In [254]:
# for index, row in transformed_laps_relative_avg_all.iterrows():
#   if transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']==0.0:
#     transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']=0.001
# transformed_laps_relative_avg_all

In [255]:
# fig=px.bar(
#     transformed_laps_relative_avg_all,
#     x="Team",
#     y="Delta_Avg_All",
#     color='Team',
#     template="xgridoff",
#     title="<b>Team Pace (Average Lap Times) for the {} {}</b>".format(year,gp),
#     text="Delta_Percent_Avg",
#     height=700, width=1200,
# color_discrete_map={
#                  "Alpine": "#0093cc",
#                  "Aston Martin": "#229971",
#                  "Ferrari": "#E80020",
#                  "Haas F1 Team": "#B6BABD",
#                  "Kick Sauber": "#52e252",
#                  "McLaren": "#FF8000",
#                  "Mercedes": "#27F4D2",
#                  "RB": "#6692FF",
#                  "Red Bull Racing": "#3671C6",
#                  "Williams": "#64C4FF" ,
#                  "Alfa Romeo":"#C92D4B",
#                  "AlphaTauri":"#5E8FAA",
#                  "Racing Point":"#F596C8",
#                  "Renault":"#FFF500",
#                  "Toro Rosso":"#469bff",
#                  "Force India":"#F596C8",
#                  "Sauber":"#9B0000"
#                  }
# )

# fig.update_traces(textposition='outside')

# fig.update_traces(opacity=0.85)

# fig.update_traces( marker_line_color='white',marker_line_width=1.5)

# fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_yaxes(range=[2, 0])

# # fig['layout']['yaxis']['autorange'] = "reversed"

# fig.update_layout(
#     xaxis={'side': 'top'},
#     yaxis={'side': 'left'}
# )

# fig.update_xaxes(title_text='')      # Remove axis titles
# fig.update_yaxes(title_text='')

# fig.update_xaxes(
#         title_standoff = 75
# )

# fig.update_layout(
#     title_x=0.5,
#     margin=dict(l=60, r=5, t=110, b=30),
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=14,
#         color="Black"
#     ),
#     title_font_family="PT Sans Narrow"
# )
# fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

# fig.show()

In [256]:
# fig.write_html("{} {} Team Avg Lap Plot.html".format(year,gp))